In [1]:
import psycopg2
import json
import string
with open("issues_final.json", 'r') as final_json_file:
    data = json.load(final_json_file)

In [3]:
host = "ec2-34-248-169-69.eu-west-1.compute.amazonaws.com"
database = "d53snj49816hrh"
user = "vxloljdzouvorj"
password = '50470fc6d490406240df9dfc17ef551598f30e86238d1f7e9ea7d766498d0a02'
port = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
            host = host,
            user = user,
            dbname = database,
            password = password,
            port = port)

    cur = conn.cursor()
    SQLscript = '''
                INSERT INTO issues (id, title, body) VALUES (%s, %s, %s)
                '''
    for current in data:
        id = current["originalId"]
        title = ' '.join(current["originalTitle"].translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))).lower().split())
        body = ' '.join(current["originalBody"].translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))).lower().split())
        values = (id, title, body)
        try:    
            cur.execute(SQLscript, values)
            conn.commit()
        except Exception as SQLerr:
            conn.rollback()
            print(SQLerr)

except Exception as error:
    print(error)

In [20]:
if cur is not None:
    cur.close()
if conn is not None:
    conn.close()

In [9]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.add("flutter")
stop_words.add("dart")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gs123\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
title = "tool crash casterror  gradlewrapper updateinner package flutter tools src flutter cache dart 545 49"
ex = title.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))).lower()
word_tokens = ex.split()
filtered = [w for w in word_tokens if not w in stop_words]

In [56]:
query = """SELECT title, ts_rank_cd(to_tsvector('english', title), query) AS rank FROM issues i, to_tsquery('english', ' """
for i, word in enumerate(filtered):
        query+= word
        if i != len(filtered) - 1:
                query+= ' | ' 
query +=  """  ') as query WHERE to_tsvector('english', title) @@  query ORDER BY RANK DESC"""

In [54]:
conn = psycopg2.connect(
            host = host,
            user = user,
            dbname = database,
            password = password,
            port = port)

cur = conn.cursor()
cur.execute(query) 
result=cur.fetchall()
conn.commit()

('tool crash casterror 0 gradlewrapper updateinner package flutter tools src flutter cache dart 538 49', 1.0)
